In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('Behavioral_Risk_Factor_Surveillance_System__BRFSS__Prevalence_Data__2011_to_present_.csv', sep = ";")

C:\Users\Yusuf\AppData\Local\Temp\ipykernel_81728\257562441.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Behavioral_Risk_Factor_Surveillance_System__BRFSS__Prevalence_Data__2011_to_present_.csv', sep = ";")


In [3]:
df['Sample_Size'] = df['Sample_Size'].str.replace(",", "")
df['Sample_Size'] = df['Sample_Size'].astype('int64')
df.rename(columns = {"Break_Out":"Category",  "Break_Out_Category":"Class_Category"}, inplace= True)


In [4]:
df['Response'] = df['Response'].replace({"Never smoked": "No", "Former smoker":"No", "Smoke some days":"Yes", "Smoke everyday":"Yes"})


In [5]:
total_sample_size = dict(df.groupby(["Year","Question","Category"])["Sample_Size"].sum())

def get_percentage(a,b,c,d):
    for year, total_number in total_sample_size.items():
        if (a, b, c) == year:
            return round((d / total_number)*100, 2)
            
def get_total_size(a,b,c):
    for year, total_number in total_sample_size.items():
        if (a,b,c) == year:
            return total_number

In [6]:
vectfunc = np.vectorize(get_percentage)
vect_get_total_size = np.vectorize(get_total_size)

In [7]:
df["Percentage"] = vectfunc(df['Year'], df['Question'], df['Category'],df['Sample_Size'])

C:\Users\Yusuf\AppData\Local\Temp\ipykernel_81728\139617235.py:6: RuntimeWarning: invalid value encountered in longlong_scalars
  return round((d / total_number)*100, 2)


In [8]:
df["Total_SS"] = vect_get_total_size(df['Year'], df['Question'], df['Category'])

In [9]:
df.loc[(df['Topic'] == "Smoker Status") & (df['Class_Category'] == "Gender")].groupby(["Year","Response","Category","Total_SS"]).aggregate({'Sample_Size': 'sum','Percentage': 'sum'}).reset_index()

,Year,Response,Category,Total_SS,Sample_Size,Percentage
0,2011,No,Female,306479,258538,84.35
1,2011,No,Male,197860,161001,81.40
2,2011,Yes,Female,306479,47941,15.67
3,2011,Yes,Male,197860,36859,18.63
4,2012,No,Female,278262,236321,84.95
5,2012,No,Male,187756,153597,81.79
6,2012,Yes,Female,278262,41941,15.11
7,2012,Yes,Male,187756,34159,18.24
8,2013,No,Female,281714,239845,85.17
9,2013,No,Male,194737,159994,82.18


In [10]:
df_final = df.groupby(["Year","Class", "Topic", "Question", "Response","Category","Class_Category", "Total_SS"]).aggregate({'Sample_Size': 'sum','Percentage': 'sum'}).reset_index()
df_final.to_csv("BRFSS.csv", index = False)


In [11]:
df.to_csv("brfss_backup.csv", index = False)

# Creating dataset for Smoker Status with additional lattitude and longitude data

In [12]:
import pandas as pd
import re
import numpy as np


In [13]:
df_smokers = pd.read_csv("brfss_backup.csv")

C:\Users\Yusuf\AppData\Local\Temp\ipykernel_81728\3591022630.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_smokers = pd.read_csv("brfss_backup.csv")


In [14]:
# Removing null values
df_smokers.drop(df_smokers[df_smokers["Locationabbr"] == "UW"].index, axis = 0, inplace = True)
df_smokers.drop(df_smokers[df_smokers["Locationabbr"] == "US"].index, axis = 0, inplace = True)

In [15]:
df_smokers = df_smokers.loc[df_smokers['Topic'] == 'Smoker Status']

In [16]:
# Creating function to seperate lat and long from Geo Location function

def get_lat(col):
    tmp = []
    for i in col:
        latlong = re.findall(r"(-?\d+.\d+)", i) 
        tmp.append(latlong[0])
    return tmp

def get_long(col):
    tmp = []
    for i in col:
        latlong = re.findall(r"(-?\d+.\d+)", i) 
        tmp.append(latlong[1])
    return tmp

df_smokers['latitude'] = get_lat(df_smokers['GeoLocation'])
df_smokers['longitude'] = get_long(df_smokers['GeoLocation'])




In [17]:
# Make the percentage of sample size according to the location
total_sample_size = dict(df_smokers.groupby(["Year","Locationdesc"])["Sample_Size"].sum())

def get_percentage(a,b,c):
    for year, total_number in total_sample_size.items():
        if (a, b) == year:
            return round((c / total_number)*100, 2)
            
def get_total_size(a,b):
    for year, total_number in total_sample_size.items():
        if (a,b) == year:
            return total_number

get_percentage_vect = np.vectorize(get_percentage)
get_total_size_vect = np.vectorize(get_total_size)

df_smokers['Percentage_Loc'] = get_percentage_vect(df_smokers['Year'], df_smokers['Locationdesc'], df_smokers['Sample_Size'])
df_smokers['Total_SS_Loc'] = get_total_size_vect(df_smokers['Year'], df_smokers['Locationdesc'])

In [18]:
df_smokers['Response']

17          No
18          No
162        Yes
357         No
476         No
          ... 
2048242    Yes
2048280     No
2048324    Yes
2048341    Yes
2048460     No
Name: Response, Length: 52551, dtype: object

In [19]:

df_smokers_to_csv = df_smokers.groupby(["Year","Locationdesc","Response","Total_SS_Loc","latitude", "longitude"]).aggregate({"Percentage_Loc":"sum", "Sample_Size":"sum"}).reset_index()


In [20]:
df_smokers_to_csv.to_csv("brfss_smokers.csv", index = False)

In [21]:
df = pd.DataFrame(
    np.random.randn(1000, 2) / [50, 50] + [37.76, -122.4],
    columns=['lat', 'lon'])

print(df)

           lat         lon
0    37.743339 -122.367431
1    37.768823 -122.397162
2    37.743169 -122.411494
3    37.775983 -122.422945
4    37.722008 -122.387931
..         ...         ...
995  37.774038 -122.392953
996  37.730793 -122.395215
997  37.769886 -122.409360
998  37.736723 -122.414874
999  37.746687 -122.382037

[1000 rows x 2 columns]
